## Run SimSS simulations and plot the results
Version 0.1
(c) Vincent M. Le Corre (https://github.com/VMLC-PV)

In [ ]:
# Activate matplotlib widgets
%matplotlib inline
# comment the next line if you are on the jupyterhub server
%matplotlib widget 
# %matplotlib notebook
# Package import
import os,platform,warnings,itertools,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from datetime import datetime
# Import homemade package by VLC
from boar.SIMsalabim_utils.MakeDevParFile import *
from boar.SIMsalabim_utils.GetInputPar import *
from boar.SIMsalabim_utils.PlotInputPar import *
from boar.SIMsalabim_utils.RunSim import PrepareSimuInputs, RunSimulation # Run simulation
from boar.SIMsalabim_utils.SimSS_plots import PlotJV, PlotJVPerf,PlotNrjDiagSimSS,PlotDensSimSS # Plotting functions
from boar.SIMsalabim_utils.CleanFolder import clean_up_output, clean_file_type # Cleaning functions
from boar.SIMsalabim_utils.aux_func import * # Plot settings
# from boar.SIMsalabim_utils import plot_settings_screen # Plot settings

# Import imeetDB
sys.path.append(os.path.join('/home/vlc/Desktop/','imeetdb')) #  absolute path to imeetDB
# Import local libraries
from imeetdb.dbconnect import *
from imeetdb.models import *
from agents.imeetDB_JV_agent import *

# Import the database session
db_session

## General Inputs
warnings.filterwarnings("ignore")           # Don't show warnings
system = platform.system()                  # Operating system
max_jobs = os.cpu_count()-2                 # Max number of parallel simulations (for number of CPU use: os.cpu_count() )
do_multiprocessing = True                      # Use multiprocessing
if system == 'Windows':                     # cannot easily do multiprocessing in Windows
        max_jobs = 1
        do_multiprocessing = False
        try:                                # kill all running jobs to avoid conflicts
            os.system('taskkill.exe /F /IM simss.exe')
        except:
            pass


# Import Experimental Data

In [ ]:
# Define the path to the data and to the SIMsalabim directory
curr_dir = os.getcwd()

# Define the path to the SIMsalabim directory and the results directory
path2simu = os.path.join('/home/vlc/Desktop/Xioyan','SIMsalabimv445_Xioyan','SimSS') # path to the SIMsalabim directory
res_dir = os.path.join(curr_dir,'temp') # path to the results directory, use this line if Notebook is in the main folder
# res_dir = os.path.join('../','temp') # path to the results directory, use this line if Notebook is in the Notebooks folder



In [ ]:
# Select username and PID of the sample
user = 'osterriedert'
PID = '20239'
pixel = 'B-1'
Gfracs2fit =[0.1,0.5,1,1.2] 
area = 0.08 *1e-4 # cm^2 * 1e-4 to get m^2
time_date = '2023-03-22 18:23:33'
#convert time_date to datetime object
time_date = datetime.strptime(time_date, '%Y-%m-%d %H:%M:%S')

### Check the stack of the device

In [ ]:
# test get stack info and process conditions
stack_info = get_stack_info_from_db(user,PID)

active_layer = []
for S0 in stack_info:
    print(S0)
    if S0['Type'] == 'active':
        for key in S0.keys():
            # check if key start with M
            if key[0] == 'M' and S0[key] != '':
                active_layer.append(S0[key])

print('Active layer:')
print(active_layer)

CB_vals = []
VB_vals = []

# full acceptor list
acceptor_list = ['Y6','Y6-BO-4F','DT-Y6','BTP-4F-12','Y12','ITIC-4F','ITIC-Th','ITIC','EH-IDTBR','O-IDTBR','O-IDFBR','IEICO-4F','PC71BM']
CBs = [4.07,4.04,4.04,4.06,4.06,4.1,3.75,3.81,3.81,3.88,3.89,4.19,4.12]

# full donor list
donor_list = ['PM6','PTQ10','PCE10','PffBT4T-2DT','D18','PDPP-5T','D4610','PTB7']
VBs = [5.5,5.55,5.23,5.24,5.56,5.14,5.2,5.15]


for i in active_layer:
    if i in acceptor_list:
        CB_vals.append(CBs[acceptor_list.index(i)])
    elif i in donor_list:
        VB_vals.append(VBs[donor_list.index(i)])
    else:
        pass

if user == 'liuc':
    CB_val = 4.04
    VB_val = 5.5
else:
    CB_val = max(CB_vals)
    VB_val = min(VB_vals)
    print( 'CB = ', CB_val, 'eV')
    print( 'VB = ', VB_val, 'eV')



### Check the structure of the device for SIMsalabim
Here we check the structure of the device for SIMsalabim and you can adjust the parameters if needed.

For example, if you want to change the thickness of the active layer (AL) you can do this by changing the value of the variable `L` in the cell below.
To do so you need to update the value of the variable ParFileDic['L'] and set MakeUpdate to True.

By convention LTL is the ETL and RTL is the HTL.\
Note that L is the **total** thickness of the ETL, HTL and active layer, so if you want to change the thickness of the active layer you need to subtract the thickness of the ETL and HTL from the total thickness. 
Such that:

$L_{AL} = L - L_{LTL} - L_{RTL}$

To check the name of the parameters in SIMsalabim please refer to the [SIMsalabim documentation](https://github.com/kostergroup/SIMsalabim).



In [ ]:
# Vizualize the stack defined for the simulation
check_SIMsalabim_input = False
if check_SIMsalabim_input:
    # Load Device parameters file and update parameters
    ParFileDic = ReadParameterFile(os.path.join(path2simu, 'device_parameters.txt')) # read the parameters from the file

    # Change parameters in the dictionary
    
    ParFileDic['L_LTL'] = 30e-9
    ParFileDic['L_RTL'] = 0e-9
    ParFileDic['L'] =  ParFileDic['L_LTL']+ParFileDic['L_RTL']+120e-9
    ParFileDic['CB'] = CB_val
    ParFileDic['VB'] = VB_val
    ParFileDic['CB_LTL'] = CB_val
    ParFileDic['W_L'] = CB_val
    ParFileDic['W_R'] = VB_val
    MakeUpdate = False # set to True to update the parameters in the file
    if MakeUpdate:
        UpdateDevParFile(ParFileDic, path2simu) # update the parameters in the file

    # Visualize the parameters
    fig, axs = plt.subplots(2,2,figsize = (10,8))
    plot_input_nrj_diag(ParFileDic,ax=axs[0, 0])
    plot_input_mob(ParFileDic,ax=axs[0, 1])
    plot_input_dens(ParFileDic,ax=axs[1, 0])
    plot_input_SRH_lifetime(ParFileDic,ax=axs[1, 1],y_unit='ns',y2_unit='cm')
    plt.tight_layout()
    plt.show()


In [ ]:
# Define function to get JV from imeetDB
def filter_DB_data(username,PID,pixel=None,time2fit=None,Gfracs=[1],area=0.08 *1e-4,Vlim=[-0.5,1],time_format='relative',time_unit='h'):
    """ Get the JVs from imeetDB and calculate some weights for the JV

    Parameters
    ----------
    username : str
        username of the user
    PID : str
        PID of the sample
    pixel : str, optional
        pixel of the sample, by default None
    time2fit : datetime or float
        time of the measurement. If time_format is 'absolute' then time is a datetime object. If time_format is 'relative' then time is a float.
    Gfracs : list, optional
        list of light intensities in suns, by default [1]
    area : float, optional
        area of the sample in m^2, by default 8e-6
    Vlim : list, optional
        voltage limits for the JV, by default [-0.5,1]
    time_format : str, optional
        time format, by default 'absolute'
        'absolute' : time is a datetime object
        'relative' : time is a float
    time_unit : str, optional
        time unit, by default 'min'
        'min' : time is in minutes
        'h' : time is in hours
        's' : time is in seconds

    Returns
    -------
    X : array
        array of the voltage and light intensity
    y : array
        array of the current
    weight : array
        array of the weights

    """    

    if pixel is None:
        pixel = get_champion_pixel_from_db(username,PID)

    if time2fit is None: # get the JV at t=0
        X,y = get_JV_from_db(username,PID,pixel,0,Gfracs=Gfracs,area=area,time_format='relative',time_unit='h')
    else:
        X,y = get_JV_from_db(username,PID,pixel,time2fit,Gfracs=Gfracs,area=area,time_format=time_format,time_unit=time_unit)

    # filter data
    idx = np.where((X[:,0] > Vlim[0]) & (X[:,0] < Vlim[1]))[0]
    X = X[idx,:]
    y = y[idx]

    # get unique values of Gfrac
    Gfrac_unique = np.unique(X[:,1])
    weight = np.ones(len(y))
    for Gfrac in Gfrac_unique:
        idx = np.where(X[:,1] == Gfrac)[0]
        V = X[idx,0]
        J = y[idx]

        if Gfrac == 0: # dark so weight = 1
            weight[idx] = 1
        else:
            power = 1/abs(J) # normalize the power to the minimum current
            for v,p in enumerate(power):
                if V[v] > 0 and J[v] < 0:
                    power[v] = 10*p
                
            weight[idx] = power  
     
        

    return X,y,weight


# Prepare the input files for SIMsalabim

In [ ]:
# Prepare the command strings to run
#First, we need to create a list of the filenames containing the experimental data

str_fit = f'-Bulk_tr 6.528993435276716e+19 -kdirect 7.498074100730517e-19 -mun_0 6.133409743633732e-07 -mup_0 1e-06 -Nc 5e+26 -Rseries 2.574826446976094e-05 -Rshunt 0.216269051812806 -W_R {5.5-0.3187421538513319} -W_L {4.06+0.007926616915289878+0.0} -CB_LTL {4.06+0.007926616915289878} -mob_LTL 2.4176269740506953e-06 -Gehp 1.3103854540615744e+28 -ETrapSingle {4.06+0.5} -CB 4.06 -VB 5.5'

str_lst = []
labels = []
for g in Gfracs2fit:
    str_lst.append('-Gfrac ' + str(g) + ' ')
    labels.append(str(g))

# Prepare necessary lists for the RunSimulation function
JV_files,Var_files,scPars_files,code_name_lst,path_lst = [],[],[],[],[]
for i in range(len(str_lst)):
    JV = 'JV_' + str(i) + '.dat'
    Var = 'Var_' + str(i) + '.dat'
    scPar = 'scPar_' + str(i) + '.dat'
    str_lst[i] = str_fit +' '+str_lst[i] + ' -JV_file ' + JV + ' -Var_file ' + Var + ' -scPars_file ' + scPar
    JV_files.append(os.path.join(path2simu,JV))
    Var_files.append(os.path.join(path2simu,Var))
    scPars_files.append(os.path.join(path2simu,scPar))
    code_name_lst.append('SimSS')
    path_lst.append(path2simu)


Simulation_Inputs = str_lst,JV_files,Var_files,scPars_files,code_name_lst,path_lst,labels

# Run the simulations
run_simu = True
if run_simu:
    RunSimulation(Simulation_Inputs,max_jobs=max_jobs,do_multiprocessing=do_multiprocessing,verbose=True) # runs the simulations


In [ ]:
# Make JV plots
# Plot the JV curves
colors = plt.cm.viridis(np.linspace(0,1,max(len(str_lst),3)+1)) # prepare color for plots

# plot lin-log JV
f = plt.figure(2,figsize=(10,10))
PlotJV(JV_files, labels=labels,colors = colors,num_fig=2,plot_type=0,x='Vext',y=['Jext'],legend=True,xlimits=[-0.2,1.2],ylimits=[-30,20],x_unit='V',y_unit='mA/cm^2',save_fig=True,fig_name=os.path.join(path2simu,'JV.png'),mark='',line_type = ['-'])

for i,g in enumerate(Gfracs2fit):
    X,y,weight = filter_DB_data(user,PID,pixel=pixel,time2fit=1,Vlim=[-0.5,1.2],Gfracs=[g],area=area,time_format='relative')
    df = pd.DataFrame({'V':X[:,0],'J':y})
 
    PlotJV(df,labels=labels,colors = colors[i],num_fig=2,plot_type=0,x='V',y=['J'],legend=False,xlimits=[-0.2,1.2],ylimits=[-30,20],x_unit='V',y_unit='mA/cm^2',save_fig=True,fig_name=os.path.join(path2simu,'JV.png'),mark='o',line_type = ['None'], data_type=1)

plt.show()


In [ ]:
# Plot performance
plot_perf = True
if plot_perf:
    PlotJVPerf(Gfracs2fit, scPars_files,y='PCE',Gfrac=Gfracs2fit,plot_type=1,xlimits=[],ylimits=[],xlabel='Light Intensity [suns]',ylabel='PCE [%]',legend='',num_fig=5,mark='o',line_type = ['-'],norm_plot=False,fig_name=os.path.join(path2simu,'JV_perf.png'))

    PCE_exp,Voc_exp,Jsc_exp,FF_exp = [],[],[],[]
    for i,g in enumerate(Gfracs2fit):
        X,y,weight = filter_DB_data(user,PID,pixel=pixel,time2fit=1,Vlim=[-0.5,1.2],Gfracs=[g],area=area,time_format='relative')
        JV = pd.DataFrame({'V':X[:,0],'J':y})
        
        PCE_exp.append(get_PCE(JV['V'],JV['J'],g)/10)
        Voc_exp.append(get_Voc(JV['V'],JV['J']))
        Jsc_exp.append(-get_Jsc(JV['V'],JV['J']))
        FF_exp.append(get_FF(JV['V'],JV['J']))

    plt.plot(Gfracs2fit,PCE_exp,'o')

    PlotJVPerf(Gfracs2fit, scPars_files,y='Voc',Gfrac=Gfracs2fit,plot_type=1,xlimits=[],ylimits=[],xlabel='Light Intensity [suns]',ylabel='V$_{OC}[V] $',legend='',num_fig=6,mark='o',line_type = ['-'],norm_plot=False,fig_name=os.path.join(path2simu,'JV_perf.png'))
    plt.plot(Gfracs2fit,Voc_exp,'o')

    PlotJVPerf(Gfracs2fit, scPars_files,y='Jsc',Gfrac=Gfracs2fit,plot_type=3,xlimits=[],ylimits=[],xlabel='Light Intensity [suns]',ylabel='J$_{SC}$[A m$^{-2}$] ',legend='',num_fig=7,mark='o',line_type = ['-'],norm_plot=False,fig_name=os.path.join(path2simu,'JV_perf.png'))
    plt.plot(Gfracs2fit,Jsc_exp,'o')

    PlotJVPerf(Gfracs2fit, scPars_files,y='FF',Gfrac=Gfracs2fit,plot_type=1,xlimits=[],ylimits=[],xlabel='Light Intensity [suns]',ylabel='FF',legend='',num_fig=8,mark='o',line_type = ['-'],norm_plot=False,fig_name=os.path.join(path2simu,'JV_perf.png'))
    plt.plot(Gfracs2fit,FF_exp,'o')



In [ ]:
# Clean output files from simulation folders
Do_Cleaning = False # Careful, this will delete all files in the folder
if Do_Cleaning:
    clean_up_output('tj',path2simu)
    clean_up_output('tVG',path2simu)
    clean_up_output('JV',path2simu)
    clean_up_output('Var',path2simu)
    clean_up_output('scPars',path2simu)